In [1]:
import pandas as pd

from pathlib import Path
repo_path = Path('/home/krajda/anticipatio/')

In [3]:
tweets = pd.read_pickle(repo_path / 'data/final.pkl')
docs = tweets['txt'].tolist()
docs

[' Zero.',
 ' Any that only needs one world.',
 'The many-worlds interpretation is the most egregious violation of Occam’s razor in history.',
 'The fight over what AIs say and do has just started, and will never end.',
 "A must-read: 's thoughts on a life in probabilistic AI.\n",
 'For better or worse, social media is where society decides what to pay attention to.',
 "If everyone who thinks they're the only anti-woke in their organization got together, they'd be the majority.",
 'What Google is singularly lacking is someone who knows how to play the PR game, like Sam Altman or Demis Hassabis (or Elon Musk or Steve Jobs).',
 ' Wokeism is least popular among Gen Z of all age cohorts, according to surveys.',
 ' Across the board.',
 ' BERT.',
 ' On the contrary. No one hates wokeism more than Gen Z.',
 ' Applying n algorithms to m datasets.',
 ' I mean BERT.',
 'Many people don’t seem to know that Google has used a large language model in its search engine for years.',
 'The ratio of new

In [8]:
doc_topic = pd.read_csv(repo_path / 'data/topics_labels.csv')
doc_topic.sort_values('doc_id',inplace=True)
tweets['Topic'] = doc_topic['topic'].tolist()


In [10]:
emotions = pd.read_pickle(repo_path / 'data/emotions.pkl')

In [11]:
tweets = tweets.reset_index()
tweets['id'] = tweets.index
emotions['id'] = emotions.index

tweets = tweets.join(emotions.set_index('id'), on='id')

In [12]:
agg = tweets.groupby('Topic').mean().drop(columns=['id', 'index'])

In [14]:
print(agg.min(), agg.mean(), agg.max())

sadness         0.017876
disgust         0.032248
anger           0.029296
pessimism       0.015706
fear            0.015630
anticipation    0.144606
surprise        0.037245
joy             0.104566
optimism        0.125887
love            0.009404
trust           0.031349
dtype: float64 sadness         0.088251
disgust         0.138668
anger           0.126473
pessimism       0.052667
fear            0.082798
anticipation    0.466254
surprise        0.089426
joy             0.501304
optimism        0.480156
love            0.071670
trust           0.129235
dtype: float64 sadness         0.249466
disgust         0.635234
anger           0.640568
pessimism       0.153579
fear            0.476167
anticipation    0.685446
surprise        0.132466
joy             0.899199
optimism        0.839367
love            0.482587
trust           0.264596
dtype: float64


In [15]:
agg.to_csv(repo_path / 'data/to_paper/emotions_per_topic.csv')

In [16]:
anticipated_representatives = []

examples = 20

for n in range(100):
    x = tweets[tweets['Topic'] == n].nlargest(examples, ['anticipation'])[['anticipation', 'txt']]
    
    a = x['anticipation'].tolist()
    t = x['txt'].tolist()

    for i in range(examples):
        anticipated_representatives.append({'topic': n, 'anticipation': a[i], 'text': t[i]})
    
    
pd.DataFrame(anticipated_representatives).to_csv(repo_path / 'data/to_paper/anticipated_representatives.csv')

In [17]:
anticipated_representatives

[{'topic': 0,
  'anticipation': 0.9184854030609131,
  'text': 'When will we see the first #AI generated hit song? Book? 🤔'},
 {'topic': 0,
  'anticipation': 0.9137517213821411,
  'text': 'Have you ever wondered what the BEST opening chess move would be for humans, aliens, and/or artificial intelligence entities to use?\n\nDiscussion of the topic here:   '},
 {'topic': 0,
  'anticipation': 0.9109010696411133,
  'text': 'So, who owns this?: \n\n has been taking a closer look at AI-generated art and is curious if #copyright law is going to get interesting in the near future.\n'},
 {'topic': 0,
  'anticipation': 0.9096344709396362,
  'text': 'What are the icons on the map stand for?\nThe Flywheel Effect on your Connect board reminded me of my current writing abt the flywheel effect in the AI assistants world. It would be super-interesting 2 explore any potential relationship between the 2 flywheels. Can u think of any?'},
 {'topic': 0,
  'anticipation': 0.9092089533805847,
  'text': "In 19

In [18]:
tweets['x'] = 1
topic_user = tweets[['Topic', 'user', 'x']].groupby(['Topic', 'user'], as_index=False)['x'].count().sort_values(by=['Topic','x'], ascending=False)

[{'topic': 0, 'tweets_count': 944, 'user': '@ipfconline1'},
 {'topic': 0, 'tweets_count': 504, 'user': '@digital_trans4m'},
 {'topic': 0, 'tweets_count': 452, 'user': '@EvanKirstel'},
 {'topic': 0, 'tweets_count': 390, 'user': 'pickover'},
 {'topic': 0, 'tweets_count': 364, 'user': '@Ronald_vanLoon'},
 {'topic': 0, 'tweets_count': 168, 'user': '@jamesvgingerich'},
 {'topic': 0, 'tweets_count': 158, 'user': '@Nicochan33'},
 {'topic': 0, 'tweets_count': 130, 'user': '@TansuYegen'},
 {'topic': 0, 'tweets_count': 124, 'user': 'scobleizer'},
 {'topic': 0, 'tweets_count': 122, 'user': '@gleonhard'},
 {'topic': 1, 'tweets_count': 1529, 'user': '@EvanKirstel'},
 {'topic': 1, 'tweets_count': 469, 'user': '@Ronald_vanLoon'},
 {'topic': 1, 'tweets_count': 403, 'user': '@jamesvgingerich'},
 {'topic': 1, 'tweets_count': 370, 'user': '@Nicochan33'},
 {'topic': 1, 'tweets_count': 341, 'user': '@FrRonconi'},
 {'topic': 1, 'tweets_count': 334, 'user': '@CurieuxExplorer'},
 {'topic': 1, 'tweets_count': 

In [21]:
topic_user[topic_user['user'] == '@EvanKirstel']['x'].sum()

77428

In [22]:
tweets['user'].value_counts(sort=True, ascending=False)[:10]

@EvanKirstel        77428
@digital_trans4m    37445
@psb_dc             36311
@KirkDBorne         33840
@jamesvgingerich    31682
@Ronald_vanLoon     30880
scobleizer          30244
@TechCrunch         29518
@ipfconline1        29341
@rwang0             26916
Name: user, dtype: int64

In [23]:
users_sum = topic_user.groupby('user')['x'].sum().reset_index()
users_sum.sort_values(by='x', ascending=False, inplace=True)
users_sum

,user,x
23,@EvanKirstel,77428
128,@digital_trans4m,37445
209,@psb_dc,36311
46,@KirkDBorne,33840
153,@jamesvgingerich,31682
...,...,...
40,@JesusSanSegundo,1
147,@ingliguori,1
361,localrat,1
6,@BigDataBRA,1


In [24]:
topic_user['share'] = topic_user.apply(lambda x: x['x'] / users_sum[users_sum['user'] == x['user']]['x'].values[0], axis=1)

In [25]:
topic_user['user_sum'] = topic_user.apply(lambda x: users_sum[users_sum['user'] == x['user']]['x'].values[0], axis=1)

In [30]:
topic_user2 = topic_user[topic_user['user_sum'] > 10]

tu2 = topic_user2.groupby('user')['share'].max().reset_index().sort_values(by='share', ascending=False)

In [45]:
topic_user2[topic_user['user'].isin(tu2[tu2['share'] > 0.5]['user'].tolist())].groupby('user').max().sort_values(by='share', ascending=False).to_csv(repo_path / 'data/to_paper/focused_users.csv')

In [35]:
tu2[tu2['share'] > 0.5]['user'].tolist()

['@peterfgallagher',
 'alphanmanas',
 'futurecheck',
 'Blazing_Science',
 'futuristufuk',
 '@timgrieveson',
 'elinafuturist',
 'meds888',
 'jeremygutsche',
 '@SBhatnagar212',
 '@gp_pulipaka',
 '@MikeTamir',
 'pickover']

In [57]:
topic_nums = tweets['Topic'].value_counts().sort_index().tolist()
topic_nums

[6389,
 7669,
 23089,
 7736,
 8911,
 4846,
 19432,
 17795,
 8143,
 12512,
 18844,
 8541,
 14372,
 18582,
 10668,
 12408,
 14864,
 21736,
 11281,
 7672,
 23252,
 9639,
 8984,
 15191,
 16281,
 15885,
 14946,
 16701,
 15015,
 16418,
 3555,
 9035,
 7723,
 14124,
 6846,
 5907,
 6746,
 15997,
 14132,
 12154,
 8891,
 6133,
 10160,
 13814,
 16168,
 6845,
 18275,
 10217,
 13549,
 9508,
 12071,
 18727,
 10213,
 24393,
 10991,
 7234,
 5434,
 12228,
 16282,
 15893,
 15868,
 8238,
 11267,
 29624,
 12833,
 4652,
 21193,
 16856,
 6938,
 15235,
 9136,
 10692,
 14661,
 5524,
 12284,
 24989,
 7034,
 4293,
 10968,
 4519,
 8260,
 10699,
 3551,
 11514,
 10722,
 11094,
 7882,
 9611,
 11630,
 9010,
 7517,
 9320,
 13828,
 4439,
 19423,
 7721,
 15122,
 9478,
 7252,
 14079]

In [79]:
key_opinion_leaders = []

examples = 10

for n in [97]:
    x = topic_user[topic_user['Topic'] == n].nlargest(examples, ['x'])[['x', 'user']]
    
    a = x['x'].tolist()
    t = x['user'].tolist()

    for i in range(examples):
        key_opinion_leaders.append(
            {
                'topic': n, 
                'user': t[i],
                'tweets_count': a[i], 
                'topic_share': a[i] / topic_nums[n], 
                'user_share': a[i] / users_sum[users_sum['user'] == t[i]]['x'].values[0]
             }
            )
        
pd.DataFrame(key_opinion_leaders).to_csv(repo_path / 'data/to_paper/key_opinion_leaders.csv')


In [77]:
topic_user[topic_user['user'] == '@gp_pulipaka'].sort_values(by='share', ascending=False)

,Topic,user,x,share,user_sum
23524,97,@gp_pulipaka,7910,0.579063,13660
14969,61,@gp_pulipaka,3911,0.286310,13660
14076,57,@gp_pulipaka,332,0.024305,13660
2081,8,@gp_pulipaka,233,0.017057,13660
9878,39,@gp_pulipaka,123,0.009004,13660
...,...,...,...,...,...
15763,65,@gp_pulipaka,1,0.000073,13660
17841,73,@gp_pulipaka,1,0.000073,13660
22840,93,@gp_pulipaka,1,0.000073,13660
21277,87,@gp_pulipaka,1,0.000073,13660


In [78]:
topic_nums[97]

9478

In [83]:
a[0]/topic_nums[97]

0.8345642540620384

In [3]:
import pandas as pd
from pathlib import Path
repo_path = Path('/home/krajda/anticipatio/')
def open_fn(f):
    try:
        return pd.read_csv(f,engine='python')
    except:
        return pd.DataFrame()

tweets2 = pd.concat([
    pd.concat(map(open_fn, Path(repo_path/'data/futurists_kol/data').rglob('*csv'))),
    pd.concat(map(open_fn, Path(repo_path/'data/futurists_rossdawson/data').rglob('*csv')))
])

tweets2.columns = ['index','user','timestamp','url','txt']
tweets2 = tweets2.drop_duplicates(subset=['txt'])
tweets2.reset_index(inplace=True,drop=True)

print(tweets2.loc[3,'txt'])

@sendavidperdue But you supported Trump... and his lies and behavior. Biiiiiiiig mistake. But, nice try..loser


In [7]:
doc2emotion = pd.read_pickle(repo_path/"data/emotions.pkl")
doc2emotion.loc[3]

sadness         0.390466
disgust         0.955950
anger           0.981388
pessimism       0.077918
fear            0.028906
anticipation    0.029112
surprise        0.016295
joy             0.020348
optimism        0.012075
love            0.005256
trust           0.005392
Name: 3, dtype: float64

In [6]:
doc2emotion

,sadness,disgust,anger,pessimism,fear,anticipation,surprise,joy,optimism,love,trust
0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0


In [9]:
from transformers import pipeline

tweets = pd.read_pickle(repo_path / "data/final.pkl")

docs = tweets["txt"].tolist()

pipe = pipeline(
    "text-classification",
    model="cardiffnlp/twitter-roberta-base-emotion-multilabel-latest",
    top_k=25,
)

pipe(docs[3])[0]

2024-02-12 12:25:38.397157: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-12 12:25:38.397247: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-12 12:25:38.397306: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-12 12:25:38.409496: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-12 12:25:39.908084: W tensorflow/c

[{'label': 'anger', 'score': 0.9813883304595947},
 {'label': 'disgust', 'score': 0.9559500813484192},
 {'label': 'sadness', 'score': 0.3904663920402527},
 {'label': 'pessimism', 'score': 0.07791759818792343},
 {'label': 'anticipation', 'score': 0.029112225398421288},
 {'label': 'fear', 'score': 0.028906209394335747},
 {'label': 'joy', 'score': 0.02034786529839039},
 {'label': 'surprise', 'score': 0.01629466377198696},
 {'label': 'optimism', 'score': 0.012074768543243408},
 {'label': 'trust', 'score': 0.005391585174947977},
 {'label': 'love', 'score': 0.00525616854429245}]

In [16]:
print(docs[3])
pipe(docs[3])[0]

But you supported Trump... and his lies and behavior. Biiiiiiiig mistake. But, nice try..loser


[{'label': 'anger', 'score': 0.9814966917037964},
 {'label': 'disgust', 'score': 0.9556466341018677},
 {'label': 'sadness', 'score': 0.3905079960823059},
 {'label': 'pessimism', 'score': 0.07687253504991531},
 {'label': 'fear', 'score': 0.029073450714349747},
 {'label': 'anticipation', 'score': 0.0288297887891531},
 {'label': 'joy', 'score': 0.020227165892720222},
 {'label': 'surprise', 'score': 0.016065144911408424},
 {'label': 'optimism', 'score': 0.011874457821249962},
 {'label': 'trust', 'score': 0.005419053602963686},
 {'label': 'love', 'score': 0.005367077421396971}]

In [14]:
print(tweets2.loc[3,'txt'])
pipe(tweets2.loc[3,'txt'])[0]

@sendavidperdue But you supported Trump... and his lies and behavior. Biiiiiiiig mistake. But, nice try..loser


[{'label': 'anger', 'score': 0.9818358421325684},
 {'label': 'disgust', 'score': 0.9528217911720276},
 {'label': 'sadness', 'score': 0.2592710554599762},
 {'label': 'pessimism', 'score': 0.06370002031326294},
 {'label': 'anticipation', 'score': 0.03384344279766083},
 {'label': 'fear', 'score': 0.026571445167064667},
 {'label': 'joy', 'score': 0.026189832016825676},
 {'label': 'surprise', 'score': 0.016805190593004227},
 {'label': 'optimism', 'score': 0.01543356291949749},
 {'label': 'trust', 'score': 0.005702258553355932},
 {'label': 'love', 'score': 0.004971153102815151}]

In [18]:
import pandas as pd
from pathlib import Path

def open_fn(f):
    try:
        return pd.read_csv(f,engine='python')
    except:
        return pd.DataFrame()

tweets2 = pd.concat([
    pd.concat(map(open_fn, Path(repo_path/'data/futurists_kol/data').rglob('*csv'))),
    pd.concat(map(open_fn, Path(repo_path/'data/futurists_rossdawson/data').rglob('*csv')))
])

tweets2.columns = ['index','user','timestamp','url','txt']
tweets2 = tweets2.drop_duplicates(subset=['txt'])
tweets2.reset_index(inplace=True,drop=True)

print(tweets2.loc[3,'txt'])

@sendavidperdue But you supported Trump... and his lies and behavior. Biiiiiiiig mistake. But, nice try..loser


In [12]:
sentiment = pd.read_csv('maciek_docs_sentiment.csv')
sentiment

,sentiment,sentiment_name
0,-1,Negative
1,1,Positive
2,-1,Negative
3,-1,Negative
4,-1,Negative
...,...,...
1458013,0,Neutral
1458014,0,Neutral
1458015,1,Positive
1458016,0,Neutral


In [13]:
tweets = tweets.reset_index()
tweets['id'] = tweets.index
sentiment['id'] = sentiment.index

tweets = tweets.join(sentiment.set_index('id'), on='id')
tweets

,index,user,timestamp,url,txt,id,sentiment,sentiment_name
0,0,alhopper,2021-02-11 14:38:49+00:00,https://twitter.com/AlHopper/status/1359874535...,@thedailybeast Are those 44 GOP Senators sayin...,0,-1,Negative
1,1,alhopper,2021-01-31 17:34:27+00:00,https://twitter.com/AlHopper/status/1355932468...,"@BarackObama Like you, he was a good man. Rega...",1,1,Positive
2,2,alhopper,2021-01-09 17:20:15+00:00,https://twitter.com/AlHopper/status/1347956363...,@RightInTheMain @MarshaBlackburn The actions a...,2,-1,Negative
3,3,alhopper,2021-01-09 17:02:52+00:00,https://twitter.com/AlHopper/status/1347951988...,@sendavidperdue But you supported Trump... and...,3,-1,Negative
4,4,alhopper,2021-01-09 11:43:13+00:00,https://twitter.com/AlHopper/status/1347871546...,"@RepDougCollins Well, my GA loser, you’re cert...",4,-1,Negative
...,...,...,...,...,...,...,...,...
1458013,1458013,zephoria,2021-01-15 22:46:15+00:00,https://twitter.com/zephoria/status/1350212732...,"This weekend, in honor of MLK, many members of...",1458013,0,Neutral
1458014,1458014,zephoria,2021-01-15 20:41:50+00:00,https://twitter.com/zephoria/status/1350181419...,"In the US, we have lost faith in science/data/...",1458014,0,Neutral
1458015,1458015,zephoria,2021-01-15 20:41:50+00:00,https://twitter.com/zephoria/status/1350181418...,I've long thought that the Office of Science/T...,1458015,1,Positive
1458016,1458016,zephoria,2021-01-15 14:47:32+00:00,https://twitter.com/zephoria/status/1350092258...,"Speakers of languages that gender cats, cars, ...",1458016,0,Neutral


In [15]:
tweets['Topic'] = maciek_labels

In [17]:
sentiment_stats = tweets.groupby('Topic').mean().drop(columns=['id', 'index'])

In [42]:
mapp = tweets.groupby('Topic', as_index=False).value_counts(subset=['sentiment_name'])

for i in range(100):
    sentiment_stats.loc[i, 'Negative'] = mapp[(mapp['Topic'] == i) & (mapp['sentiment_name'] == 'Negative')]['count'].values[0]
    sentiment_stats.loc[i, 'Neutral'] = mapp[(mapp['Topic'] == i) & (mapp['sentiment_name'] == 'Neutral')]['count'].values[0]
    sentiment_stats.loc[i, 'Positive'] = mapp[(mapp['Topic'] == i) & (mapp['sentiment_name'] == 'Positive')]['count'].values[0]
    


In [46]:
sentiment_stats = sentiment_stats.rename(columns={'sentiment':'mean_sentiment'})

In [48]:
sentiment_stats['emotional_ratio'] = sentiment_stats.apply(lambda x: (x['Positive'] + x['Negative'])/(x['Positive'] + x['Negative'] + x['Neutral']), axis=1)

In [54]:
sentiment_stats.to_csv('sentiment_stats.csv')


In [51]:
sentiment_stats['pos_neg_ratio'] = sentiment_stats.apply(lambda x: (x['Positive'])/(x['Positive'] + x['Negative']), axis=1)

In [52]:
sentiment_stats['neg_pos_ratio'] = sentiment_stats.apply(lambda x: (x['Negative'])/(x['Positive'] + x['Negative']), axis=1)

In [55]:
sentiment_stats

,mean_sentiment,Negative,Neutral,Positive,emotional_ratio,pos_neg_ratio,neg_pos_ratio
Topic,,,,,,,
0,0.095332,301.0,6259.0,1024.0,0.174710,0.772830,0.227170
1,0.171647,431.0,7704.0,2206.0,0.255004,0.836557,0.163443
2,0.806105,524.0,3494.0,19407.0,0.850843,0.973709,0.026291
3,0.245552,402.0,7338.0,3052.0,0.320052,0.883613,0.116387
4,-0.000571,984.0,10301.0,977.0,0.159925,0.498215,0.501785
...,...,...,...,...,...,...,...
95,0.045556,59.0,8011.0,447.0,0.059411,0.883399,0.116601
96,0.048076,242.0,14386.0,993.0,0.079060,0.804049,0.195951
97,0.147787,14.0,8097.0,1423.0,0.150724,0.990257,0.009743
